In [ ]:
import pandas as pd
import requests
import json
from dateutil import parser
from dateutil.relativedelta import relativedelta
from datetime import datetime
import matplotlib.pyplot as plt
import json
import matplotlib

try:
    from tkinter.filedialog import askopenfilename
except:
    pass

import html2text
import warnings
from timezonefinder import TimezoneFinder

In [ ]:
import nbimporter
from use_schema import apply_schema

In [ ]:
%matplotlib inline

# Functions

In [ ]:
def csv_file_import(path, lat, lon):
    df, schema = apply_schema(path)
    units = schema['units']


    print(f'Lat: {lat}, Lon: {lon}')
    tf = TimezoneFinder()
    timezone_str = tf.timezone_at(lat=lat, lng=lon)
    assert timezone_str
    print(f'Timezone: {timezone_str}')
    
    # convert local time to UTC
    df = (df.tz_localize(
                    timezone_str,
                    ambiguous='NaT',
                    nonexistent='shift_forward') #add local offset
    #add local offset
                .tz_convert('UTC') #convert to UTC offset
                .tz_localize(
                    tz=None,
                    ambiguous='infer',
                    nonexistent='shift_forward'
                    )) #remove offset
    return df, units

In [ ]:
def handle_units(load, units):
    """
    Accept load in a range of units.
    Return load in kWh, sampled at 1 hour
    """
    multiplier = {'kW':10**3, 'kWh':10**3, 'MW':10*6, 'MWh':10**6, 'W':1}
    
    if not units in multiplier:
        raise KeyError('Could not find units of power or energy')

    load = (load * multiplier[units]) / 10**3 # load now in kW
    
    warnings.warn('Need to fix energy integration when resampling', RuntimeWarning)
    
    if 'h' not in units:
        print('Need to integrate power to get energy')
        load = load.resample('1h').mean()
    else:
        load = load.resample('1h').sum()
        
    return load.rename(columns={units:'kWh'})

In [ ]:
def api(dict_input, file):
    # dict_input = json.loads(json_input)

    assert file is not None
    for k, v in dict_input.items():
        assert v is not None

    lat = float(dict_input['lat'])
    lon = float(dict_input['lon'])
    cost_per_kWp= float(dict_input['cost_per_kWp'])
    import_cost=float(dict_input['import_cost'])
    export_price=float(dict_input['export_price'])
    expected_life_yrs=float(dict_input['expected_life_yrs'])
    roof_size_m2=float(dict_input['roof_size_m2'])
    azimuth=float(dict_input['azimuth'])
    roofpitch=float(dict_input['roofpitch'])
    
    
#     roof_size_m2 = 50
    panel_efficiency = 0.18 # https://www.solar.com/learn/solar-panel-efficiency/
    roof_size_kw = roof_size_m2 * panel_efficiency # STC 1000 W/m^2 @ 25 deg C
    
    print(f'Maximum system size according to roof dimensions: {roof_size_kw} kWp')


#     load = orxa_PDS_import(file)
    load, units = csv_file_import(file, lat, lon)
    print(f'Units: {units}')
    
    load = handle_units(load, units)
    
    return load

# TESTS

In [ ]:
form_data = {
    'load_data_path' : 'Factory_Heavy_loads_15min.csv',
    'lat' : 18.640976,
    'lon' : 73.833332,
    'cost_per_kWp' : 1840,
    'import_cost' : 0.14,
    'export_price' : 0.04,
    'expected_life_yrs' : 20,
    'roof_size_m2' : 250,
    'azimuth' : 180,
    'roofpitch' : 45
}

In [ ]:
file = askopenfilename(initialdir='../..')

In [ ]:
files = file

In [ ]:
files

In [ ]:
for file in files:
    print(file)

In [ ]:
load = api(form_data, file)

In [ ]:
load